7. In Sections 5.3.2 and 5.3.3, we saw that the `cv.glm()` function can be used in order to compute the LOOCV test error estimate. 
Alternatively, one could compute those quantities using just the `glm()` and `predict.glm()` functions, and a for loop. 
You will now take this approach in order to compute the LOOCV error for a simple logistic regression model on the `Weekly` data set. 
Recall that in the context of classification problems, the LOOCV error is given in (5.4).

In [2]:
library(ISLR2)
library(MASS)
library(ggplot2)
library(ggthemes)
library(boot)
library(skimr)
library(caret)

# Define a color cycle to use
colors <- colorblind_pal()(8)


(a) Fit a logistic regression model that predicts `Direction` using `Lag1` and `Lag2`.

In [4]:
head(Weekly)

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,1990,0.816,1.572,-3.936,-0.229,-3.484,0.1549760,-0.270,Down
2,1990,-0.270,0.816,1.572,-3.936,-0.229,0.1485740,-2.576,Down
3,1990,-2.576,-0.270,0.816,1.572,-3.936,0.1598375,3.514,Up
4,1990,3.514,-2.576,-0.270,0.816,1.572,0.1616300,0.712,Up
5,1990,0.712,3.514,-2.576,-0.270,0.816,0.1537280,1.178,Up
6,1990,1.178,0.712,3.514,-2.576,-0.270,0.1544440,-1.372,Down


In [5]:
str(Weekly)

'data.frame':	1089 obs. of  9 variables:
 $ Year     : num  1990 1990 1990 1990 1990 1990 1990 1990 1990 1990 ...
 $ Lag1     : num  0.816 -0.27 -2.576 3.514 0.712 ...
 $ Lag2     : num  1.572 0.816 -0.27 -2.576 3.514 ...
 $ Lag3     : num  -3.936 1.572 0.816 -0.27 -2.576 ...
 $ Lag4     : num  -0.229 -3.936 1.572 0.816 -0.27 ...
 $ Lag5     : num  -3.484 -0.229 -3.936 1.572 0.816 ...
 $ Volume   : num  0.155 0.149 0.16 0.162 0.154 ...
 $ Today    : num  -0.27 -2.576 3.514 0.712 1.178 ...
 $ Direction: Factor w/ 2 levels "Down","Up": 1 1 2 2 2 1 2 2 2 1 ...


In [6]:
skim(Weekly)

── Data Summary ────────────────────────
                           Values
Name                       Weekly
Number of rows             1089  
Number of columns          9     
_______________________          
Column type frequency:           
  factor                   1     
  numeric                  8     
________________________         
Group variables            None  

── Variable type: factor ───────────────────────────────────────────────────────
  skim_variable n_missing complete_rate ordered n_unique top_counts       
1 Direction             0             1 FALSE          2 Up: 605, Dow: 484

── Variable type: numeric ──────────────────────────────────────────────────────
  skim_variable n_missing complete_rate     mean    sd        p0      p25
1 Year                  0             1 2000.     6.03 1990      1995    
2 Lag1                  0             1    0.151  2.36  -18.2      -1.15 
3 Lag2                  0             1    0.151  2.36  -18.2      -1.15 
4 Lag3   

In [7]:
lr_fit <- glm(Direction ~ Lag1 + Lag2, data = Weekly, family = "binomial")
summary(lr_fit)


Call:
glm(formula = Direction ~ Lag1 + Lag2, family = "binomial", data = Weekly)

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-1.623  -1.261   1.001   1.083   1.506  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  0.22122    0.06147   3.599 0.000319 ***
Lag1        -0.03872    0.02622  -1.477 0.139672    
Lag2         0.06025    0.02655   2.270 0.023232 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1496.2  on 1088  degrees of freedom
Residual deviance: 1488.2  on 1086  degrees of freedom
AIC: 1494.2

Number of Fisher Scoring iterations: 4


(b) Fit a logistic regression model that predicts `Direction` using `Lag1` and `Lag2` using all but the first observation.

In [8]:
lr_fit_2 <- glm(
    Direction ~ Lag1 + Lag2,
    data = Weekly,
    subset = 2:nrow(Weekly),
    family = "binomial"
)
summary(lr_fit_2)


Call:
glm(formula = Direction ~ Lag1 + Lag2, family = "binomial", data = Weekly, 
    subset = 2:nrow(Weekly))

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.6258  -1.2617   0.9999   1.0819   1.5071  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  0.22324    0.06150   3.630 0.000283 ***
Lag1        -0.03843    0.02622  -1.466 0.142683    
Lag2         0.06085    0.02656   2.291 0.021971 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1494.6  on 1087  degrees of freedom
Residual deviance: 1486.5  on 1085  degrees of freedom
AIC: 1492.5

Number of Fisher Scoring iterations: 4


(c) Use the model from (b) to predict the direction of the first observation. 
You can do this by predicting that the first observation will go up if $P($`Direction = "Up" | Lag1, Lag2`$) > 0.5$. Was this observation correctly classified?

In [12]:
lr_probs_2 <- predict.glm(
    lr_fit_2,
    data.frame(Lag1 = Weekly$Lag1[1], Lag2 = Weekly$Lag2[1]),
    type = "response"
)
lr_probs_2

1 
0.5713923

In [11]:
lr_pred_2 <- ifelse(lr_probs_2 > 0.5, "Up", "Down")
lr_pred_2

1 
"Up"

In [24]:
lr_pred_2 == Weekly$Direction[1]

[1] FALSE

The LOOCV method misclassified the first observation.

(d) Write a `for` loop from $ i = 1 \text{ to } n$, where $n$ is the number of observations in the data set, that performs each of the following steps:

  1. Fit a logistic regression model using all but the $i$-th observation to predict `Direction` using `Lag1` and `Lag2`.
  
  2. Compute the posterior probability of the market moving up for the $i$-th observation.
  
  3. Use the posterior probability for the $i$-th observation in order to predict whether or not the market moves up.
  
  4. Determine whether or not an error was made in predicting the direction for the $i$-th observation. 
  If an error was made, then indicate this as a 1, and otherwise indicate it as a 0.

In [25]:
num_rows <- nrow(Weekly)
cv_error <- numeric(length = num_rows)
seq_num_rows <- seq(num_rows)
for (i in seq_num_rows) {
    fit <- glm(
        Direction ~ Lag1 + Lag2,
        data = Weekly,
        subset = seq_num_rows[-i],
        family = "binomial"
    )
    probs <- predict.glm(
        fit,
        data.frame(Lag1 = Weekly$Lag1[i], Lag2 = Weekly$Lag2[i]),
        type = "response"
    )
    pred <- ifelse(probs > 0.5, "Up", "Down")
    cv_error[i] <- as.numeric(pred != Weekly$Direction[i])
}

(e) Take the average of the $n$ numbers obtained in (d)iv in order to obtain the LOOCV estimate for the test error. Comment on the results.

In [26]:
mean(cv_error)

[1] 0.4499541

The overall error for the LOOCV is about 45%. This means that the LOOCV misclassified the market predictions in almost half the times, i.e., almost similar to chance.